In [1]:
import pandas as pd
import numpy as np
import csv
import os
import hdbscan
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
embeddings = pd.read_csv("embeddings.csv")#.groupby(["name", "start", "end"]).min()
embeddings

,Unnamed: 0,name,start,end,0,1,2,3,4,5,...,2424,2425,2426,2427,2428,2429,2430,2431,2432,2433
0,0,Lipaugus-vociferans-101,0.0,3.0,-13.816298,-14.779047,-16.604600,-15.838810,-15.027637,-15.056735,...,-12.145511,-11.115162,-12.262828,-16.463829,-14.977942,-14.783653,-17.306732,-17.344374,-15.889820,-18.359724
1,1,Lipaugus-vociferans-101,3.0,6.0,-18.295492,-23.361988,-16.476677,-16.182661,-20.370573,-18.820126,...,-17.087452,-16.139894,-18.203566,-19.646124,-16.419035,-19.220581,-21.260557,-20.855955,-21.188318,-21.006466
2,2,Lipaugus-vociferans-101,6.0,9.0,-16.988670,-14.263780,-15.598515,-13.066281,-18.128311,-17.151094,...,-13.035241,-12.997391,-13.668798,-18.122746,-13.178307,-16.603168,-16.691458,-17.818018,-15.466855,-18.919224
3,3,Lipaugus-vociferans-101,9.0,12.0,-18.860504,-15.989571,-14.156415,-17.250800,-15.416119,-18.675543,...,-16.133678,-11.821482,-14.627608,-20.467419,-18.575445,-18.541843,-17.874775,-18.356724,-15.991284,-20.168705
4,4,Lipaugus-vociferans-101,12.0,15.0,-16.601377,-13.814850,-16.925186,-16.395187,-14.415050,-16.172974,...,-14.503489,-11.006583,-13.393004,-18.896938,-17.942800,-15.649605,-18.209840,-16.827469,-15.904139,-22.148605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,4407,XC425555 - Screaming Piha - Lipaugus vociferans,9.0,12.0,-18.013618,-24.863628,-16.931587,-19.127516,-20.027803,-17.773487,...,-19.206300,-15.002372,-14.268598,-24.302435,-20.110710,-16.916174,-16.110807,-19.629211,-20.102268,-23.995580
4408,4408,XC425555 - Screaming Piha - Lipaugus vociferans,12.0,15.0,-21.499010,-25.396652,-16.560404,-17.054680,-22.986364,-21.447160,...,-18.272898,-17.639322,-19.193445,-20.472080,-20.916980,-23.617400,-24.278397,-24.127690,-22.998655,-25.541506
4409,4409,XC552343 - Screaming Piha - Lipaugus vociferans,0.0,3.0,-16.607243,-18.708727,-10.991514,-16.958113,-14.564127,-12.520323,...,-15.605555,-14.048874,-15.532355,-18.907902,-16.524933,-18.902752,-17.739126,-19.028685,-17.991568,-19.552902
4410,4410,XC552343 - Screaming Piha - Lipaugus vociferans,3.0,6.0,-12.631995,-17.275229,-12.096788,-15.733994,-15.280943,-13.279224,...,-14.526381,-15.870881,-16.013033,-19.354643,-13.932918,-20.362356,-15.766384,-17.986895,-16.208982,-19.552298


In [3]:
classifications_no_duplicates = pd.read_csv("./classifications_no_duplicates.csv")#.groupby(["Filename", 'Begin Time (s)', 'End Time (s)']).min()
classifications_no_duplicates

,Unnamed: 0,Filename,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
0,4704,Lipaugus-vociferans-73778,1,Spectrogram 1,1,12.0,15.0,150,12000,scrpih1,Screaming Piha,1.0000
1,3030,Lipaugus-vociferans-443978,38,Spectrogram 1,1,192.0,195.0,150,12000,scrpih1,Screaming Piha,1.0000
2,2947,Lipaugus-vociferans-442394,8,Spectrogram 1,1,117.0,120.0,150,12000,scrpih1,Screaming Piha,1.0000
3,3012,Lipaugus-vociferans-443978,20,Spectrogram 1,1,147.0,150.0,150,12000,scrpih1,Screaming Piha,0.9999
4,4281,Lipaugus-vociferans-542536,3,Spectrogram 1,1,24.0,27.0,150,12000,scrpih1,Screaming Piha,0.9999
...,...,...,...,...,...,...,...,...,...,...,...,...
4038,4289,Lipaugus-vociferans-543334,6,Spectrogram 1,1,24.0,27.0,150,12000,humnov,Human non-vocal,0.1013
4039,1499,Lipaugus-vociferans-276152,2,Spectrogram 1,1,15.0,18.0,150,12000,leswoo2,Lesser Woodcreeper,0.1012
4040,1612,Lipaugus-vociferans-301423,9,Spectrogram 1,1,24.0,27.0,150,12000,scrpih1,Screaming Piha,0.1011
4041,3143,Lipaugus-vociferans-450616,4,Spectrogram 1,1,24.0,27.0,150,12000,gohman1,Golden-headed Manakin,0.1008


In [4]:
embed_and_class = embeddings.merge(classifications_no_duplicates, left_on=["name", "start", "end"], right_on=["Filename", "Begin Time (s)",  "End Time (s)"])
embed_and_class

,Unnamed: 0_x,name,start,end,0,1,2,3,4,5,...,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
0,1,Lipaugus-vociferans-101,3.0,6.0,-18.295492,-23.361988,-16.476677,-16.182661,-20.370573,-18.820126,...,9,Spectrogram 1,1,3.0,6.0,150,12000,scrpih1,Screaming Piha,0.9818
1,2,Lipaugus-vociferans-101,6.0,9.0,-16.988670,-14.263780,-15.598515,-13.066281,-18.128311,-17.151094,...,12,Spectrogram 1,1,6.0,9.0,150,12000,scrpih1,Screaming Piha,0.5652
2,3,Lipaugus-vociferans-101,9.0,12.0,-18.860504,-15.989571,-14.156415,-17.250800,-15.416119,-18.675543,...,13,Spectrogram 1,1,9.0,12.0,150,12000,undtin1,Undulated Tinamou,0.1768
3,4,Lipaugus-vociferans-101,12.0,15.0,-16.601377,-13.814850,-16.925186,-16.395187,-14.415050,-16.172974,...,1,Spectrogram 1,1,12.0,15.0,150,12000,blfant1,Black-faced Antthrush,0.3000
4,5,Lipaugus-vociferans-101,15.0,18.0,-17.763756,-16.216347,-15.928372,-15.976003,-17.659664,-17.366117,...,2,Spectrogram 1,1,15.0,18.0,150,12000,scrpih1,Screaming Piha,0.1527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,4407,XC425555 - Screaming Piha - Lipaugus vociferans,9.0,12.0,-18.013618,-24.863628,-16.931587,-19.127516,-20.027803,-17.773487,...,5,Spectrogram 1,1,9.0,12.0,150,12000,spigua1,Spix's Guan,0.9079
4039,4408,XC425555 - Screaming Piha - Lipaugus vociferans,12.0,15.0,-21.499010,-25.396652,-16.560404,-17.054680,-22.986364,-21.447160,...,2,Spectrogram 1,1,12.0,15.0,150,12000,scrpih1,Screaming Piha,0.9921
4040,4409,XC552343 - Screaming Piha - Lipaugus vociferans,0.0,3.0,-16.607243,-18.708727,-10.991514,-16.958113,-14.564127,-12.520323,...,1,Spectrogram 1,1,0.0,3.0,150,12000,scrpih1,Screaming Piha,0.9997
4041,4410,XC552343 - Screaming Piha - Lipaugus vociferans,3.0,6.0,-12.631995,-17.275229,-12.096788,-15.733994,-15.280943,-13.279224,...,2,Spectrogram 1,1,3.0,6.0,150,12000,scrpih1,Screaming Piha,0.9971


In [6]:
display(embed_and_class)

,Unnamed: 0_x,name,start,end,0,1,2,3,4,5,...,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
0,1,Lipaugus-vociferans-101,3.0,6.0,-18.295492,-23.361988,-16.476677,-16.182661,-20.370573,-18.820126,...,9,Spectrogram 1,1,3.0,6.0,150,12000,scrpih1,Screaming Piha,0.9818
1,2,Lipaugus-vociferans-101,6.0,9.0,-16.988670,-14.263780,-15.598515,-13.066281,-18.128311,-17.151094,...,12,Spectrogram 1,1,6.0,9.0,150,12000,scrpih1,Screaming Piha,0.5652
2,3,Lipaugus-vociferans-101,9.0,12.0,-18.860504,-15.989571,-14.156415,-17.250800,-15.416119,-18.675543,...,13,Spectrogram 1,1,9.0,12.0,150,12000,undtin1,Undulated Tinamou,0.1768
3,4,Lipaugus-vociferans-101,12.0,15.0,-16.601377,-13.814850,-16.925186,-16.395187,-14.415050,-16.172974,...,1,Spectrogram 1,1,12.0,15.0,150,12000,blfant1,Black-faced Antthrush,0.3000
4,5,Lipaugus-vociferans-101,15.0,18.0,-17.763756,-16.216347,-15.928372,-15.976003,-17.659664,-17.366117,...,2,Spectrogram 1,1,15.0,18.0,150,12000,scrpih1,Screaming Piha,0.1527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,4407,XC425555 - Screaming Piha - Lipaugus vociferans,9.0,12.0,-18.013618,-24.863628,-16.931587,-19.127516,-20.027803,-17.773487,...,5,Spectrogram 1,1,9.0,12.0,150,12000,spigua1,Spix's Guan,0.9079
4039,4408,XC425555 - Screaming Piha - Lipaugus vociferans,12.0,15.0,-21.499010,-25.396652,-16.560404,-17.054680,-22.986364,-21.447160,...,2,Spectrogram 1,1,12.0,15.0,150,12000,scrpih1,Screaming Piha,0.9921
4040,4409,XC552343 - Screaming Piha - Lipaugus vociferans,0.0,3.0,-16.607243,-18.708727,-10.991514,-16.958113,-14.564127,-12.520323,...,1,Spectrogram 1,1,0.0,3.0,150,12000,scrpih1,Screaming Piha,0.9997
4041,4410,XC552343 - Screaming Piha - Lipaugus vociferans,3.0,6.0,-12.631995,-17.275229,-12.096788,-15.733994,-15.280943,-13.279224,...,2,Spectrogram 1,1,3.0,6.0,150,12000,scrpih1,Screaming Piha,0.9971
